<a href="https://colab.research.google.com/github/gtzambranop/UN_2020_2/blob/DCD/DCD/module_2_francisco/Taller_3_Python_b%C3%A1sico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Taller III

- Previos:
    - Descargar el dataset de iris y el dataset de iris con datos faltantes (siguiente diapositiva).
    - Cargar ambos datasets en jupyter o colab respectivamente
- Impute el dataset de faltantes por los siguientes métodos:
    - Media.
    - Médiana.
    - KNN.
    - Mice.
- Con el dataset completo y el de faltantes, calcule la sumatoria de las diferencias absolutas elemento a elemento para ver suál da mejores resultados (El de menor sumatoria).
- Determine posibles outliers univariados sobre las 4 primeras columnas del dataset iris completo ppr los siguientes métodos:
    - Basado en RI (Rango intercuartílico).
    - Estadística Z o Z-score.

In [ ]:
# Instalación de librerías faltantes
! pip install impyute

In [ ]:
# Carga de librerías necesarias para los ejercicios

import pandas as pd
import numpy as np
from impyute.imputation.cs import fast_knn, mice # Se importan los dos métodos en una sola línea
from scipy import stats # Stats es usado para calcular el Z-score

In [ ]:
# Descarga de conjunto de datos de la red
! wget https://www.dropbox.com/s/wq48iaeeahwqau4/Iris_faltantes.xlsx
! wget https://www.dropbox.com/s/xz2feavbubiu6dw/Iris.xlsx

--2020-09-27 05:45:01--  https://www.dropbox.com/s/wq48iaeeahwqau4/Iris_faltantes.xlsx
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.1, 2620:100:601d:1::a27d:501
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/wq48iaeeahwqau4/Iris_faltantes.xlsx [following]
--2020-09-27 05:45:01--  https://www.dropbox.com/s/raw/wq48iaeeahwqau4/Iris_faltantes.xlsx
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4d07bfceee7f9a8b29f4bc9b0c.dl.dropboxusercontent.com/cd/0/inline/BAKBy0-IkmSN6wnRa1piql7R6UBsXoo1VI6J1tgFQWZqIHMsXwIuKe3UvOQ85okWTy7YZCpRR8TMpC2xtC1VzgSDBDN1DVGhVWhsYpleCGN506PV3yR2Oi6XBjfJNY2C7F4/file# [following]
--2020-09-27 05:45:02--  https://uc4d07bfceee7f9a8b29f4bc9b0c.dl.dropboxusercontent.com/cd/0/inline/BAKBy0-IkmSN6wnRa1piql7R6UBsXoo1VI6J1tgFQWZqIHMsXwIuKe3UvOQ85okWTy7YZCpRR8TMpC2xtC1VzgSDBDN1D

In [ ]:
# Carga de conjunto de datos en notebook

df_iris = pd.read_excel('Iris.xlsx')
df_iris_faltantes = pd.read_excel('Iris_faltantes.xlsx')

In [ ]:
# Una vista previa del conjunto de datos 

df_iris_faltantes.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,NaN,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,NaN,Iris-setosa


## Ejercicio 1

- Impute el dataset de faltantes por los siguientes métodos:
    - Media.
    - Mediana.
    - KNN.
    - Mice.
- Con el dataset completo y el de faltantes, calcule la sumatoria de las diferencias absolutas elemento a elemento para ver suál da mejores resultados (El de menor sumatoria).

### Versión 1 (manual)

In [ ]:
# Imputación por la media
df_iris_mean = df_iris_faltantes.copy()
dc_mean = df_iris_mean.mean().to_dict()
df_iris_mean = df_iris_mean.fillna(dc_mean)

# Imputación por la mediana
df_iris_median = df_iris_faltantes.copy()
dc_median = df_iris_median.mean().to_dict()
df_iris_median = df_iris_median.fillna(dc_median)

# Extracción de solo las variables númericas (esto se usa para los métodos KNN y mice)
df_iris_numeric = df_iris_faltantes[df_iris_faltantes.columns[0:4]]

# Imputación por KNN
df_iris_knn = fast_knn(df_iris_numeric.values, k=10) 
df_iris_knn = pd.DataFrame(df_iris_knn)
# Se añade la columna clase previamente removida
df_iris_knn = pd.concat([df_iris_knn, df_iris_faltantes['class']], axis=1)
df_iris_knn.columns=df_iris_faltantes.columns

# Imputación por MICE
df_iris_mice = mice(df_iris_numeric.values) 
df_iris_mice = pd.DataFrame(df_iris_mice)
# Se añade la columna clase previamente removida
df_iris_mice = pd.concat([df_iris_mice, df_iris_faltantes['class']], axis=1)
df_iris_mice.columns=df_iris_faltantes.columns

In [ ]:
# Se extraen solo las columnas numéricas para comparar
iris_mean_values = df_iris_mean[df_iris_mean.columns[0:4]].values
iris_median_values = df_iris_median[df_iris_median.columns[0:4]].values
iris_knn_values = df_iris_knn[df_iris_knn.columns[0:4]].values
iris_mice_values = df_iris_mice[df_iris_mice.columns[0:4]].values
iris_values = df_iris[df_iris.columns[0:4]].values

# Se calcula la suma de las firerencias absolutas
sum_diff_mean = np.sum(np.abs(iris_mean_values -iris_values))
sum_diff_median = np.sum(np.abs(iris_median_values -iris_values))
sum_diff_knn = np.sum(np.abs(iris_knn_values -iris_values))
sum_diff_mice = np.sum(np.abs(iris_mice_values -iris_values))

print('La sumatoria de los valores absolutos para las diferencias con el método mean es: {}'.format(sum_diff_mean))
print('La sumatoria de los valores absolutos para las diferencias con el método median es: {}'.format(sum_diff_median))
print('La sumatoria de los valores absolutos para las diferencias con el método knn es: {}'.format(sum_diff_knn))
print('La sumatoria de los valores absolutos para las diferencias con el método mice es: {}'.format(sum_diff_mice))

La sumatoria de los valores absolutos para las diferencias con el método mean es: 47.2874556399552
La sumatoria de los valores absolutos para las diferencias con el método median es: 47.2874556399552
La sumatoria de los valores absolutos para las diferencias con el método knn es: 38.19359062077723
La sumatoria de los valores absolutos para las diferencias con el método mice es: 9.502411601716826


In [ ]:
def imputation_methods(df, type_imput):
  # Si el método de imputación es mean
  if type_imput == 'mean':
    dc_mean = df.mean().to_dict()
    df_imputated = df.fillna(dc_mean)
  # Si el método de imputación es median
  elif type_imput == 'median':
    dc_median = df.median().to_dict()
    df_imputated = df.fillna(dc_median)

  # Los métodos de imputación mice y knn demandan el uso de variables númericas
  # por esto debe haber un pre procesamiento para eliminar la última columna  y
  # posterior a la imputación restaurar dicha columna
  elif type_imput in ('knn', 'mice'):

    # Extración solo de variables númericas (cómún a mice y knn)
    df_iris_numeric = df[df.columns[0:4]] 
    
    # Imputación por knn
    if type_imput == 'knn':
      df_imputated = fast_knn(df_iris_numeric.values, k=10) 
    # Imputación por mice
    elif type_imput == 'mice':
      df_imputated = mice(df_iris_numeric.values, k=10) 

    # Transformación posterior a la imputació y restauración de la columna removida, esto aplica para mice y knn
    # Transformar df_imputated nuevamente en dataframe
    df_imputated = pd.DataFrame(df_imputated)
    # Se añade la columna clase previamente removida
    df_imputated = pd.concat([df_imputated, df['class']], axis=1)
    df_imputated.columns=df.columns
    
  return df_imputated

In [ ]:
# Lista de los tipos de imputación para iterar
lst_type_imputations = ['mean', 'median', 'knn', 'mice']

# Diccionario para almacenar los datasets imputados
dc_imputed_datasets = {}

# Se itera la lista de los métodos a imputar
for type_imputation in lst_type_imputations:

  # EL resultado de la imputación (llamado a función) se almacena en un diccionario de la forma {llave: valor}
  # donde llave el el método usado y valor es la tabla después de la imputación
  dc_imputed_datasets[type_imputation] = imputation_methods(df_iris_faltantes, type_imputation)

In [ ]:
# Diccionario para almacenar las diferencias
dc_sum_diff_abs = {}

# Se itera el diccionario que tiene los resultados de las imputaciones
for type_imputation, df_imputed in dc_imputed_datasets.items():
  # Se extraen solo las columnas numéricas para comparar
  df_imputed_numeric = df_imputed[df_imputed.columns[0:4]]
  df_iris_numeric = df_iris[df_iris.columns[0:4]]

  # Se calcula la diferencia elemento a elemento y posteriormente se obtiene el valor absoluto
  diff_abs = np.abs(df_imputed_numeric.values - df_iris_numeric.values)

  # Se almacena la sumatoria de las diferencias en un diccionario
  dc_sum_diff_abs[type_imputation] = np.sum(diff_abs)  

dc_sum_diff_abs

{'knn': 38.19359062077723,
 'mean': 47.2874556399552,
 'median': 44.599999999999994,
 'mice': 9.879595734706315}

- Determine posibles outliers univariados sobre las 4 primeras columnas del dataset iris completo ppr los siguientes métodos:
    - Basado en RI (Rango intercuartílico).
    - Estadística Z o Z-score.

### Rango intercuartílico

In [ ]:
ri = df_iris.quantile(.75) - df_iris.quantile(.25)
ls = df_iris.quantile(.75)  + 1.5 * ri
li = df_iris.quantile(.25)  - 1.5 * ri
df_iris.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class'], dtype='object')

In [ ]:
for col in ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']:
  print('-'*80)
  print('Los límites superior e inferior pala la columna {} son: {} y {}'.format(col, ls[col], li[col]))
  print('El sub-dataframe con los outliers para la columna {} es el siguiente:'.format(col))
  bool_slice = np.logical_or(df_iris[col] < li[col], df_iris[col] > ls[col])
  print(df_iris[bool_slice])



--------------------------------------------------------------------------------
Los límites superior e inferior pala la columna sepal_length son: 8.350000000000001 y 3.1499999999999986
El sub-dataframe con los outliers para la columna sepal_length es el siguiente:
Empty DataFrame
Columns: [sepal_length, sepal_width, petal_length, petal_width, class]
Index: []
--------------------------------------------------------------------------------
Los límites superior e inferior pala la columna sepal_width son: 4.05 y 2.05
El sub-dataframe con los outliers para la columna sepal_width es el siguiente:
    sepal_length  sepal_width  petal_length  petal_width            class
15           5.7          4.4           1.5          0.4      Iris-setosa
32           5.2          4.1           1.5          0.1      Iris-setosa
33           5.5          4.2           1.4          0.2      Iris-setosa
60           5.0          2.0           3.5          1.0  Iris-versicolor
------------------------------

### Z-Score

In [ ]:
# Se extraen solo las columnas numéricas
df_iris_numeric = df_iris[df_iris.columns[0:4]]

# Se estandariza o se normaliza el conjunto de datos
z_score = pd.DataFrame(stats.zscore(df_iris_numeric))

# Se calcula el valor máximo de la estadística Z
norm = stats.norm()
max_val = norm.ppf(.995)
#max_val = 3

# Se compara el conjunto de datos estandarizados con el valo máximo creando una matriz de booleanos
df_bool = z_score.abs() > max_val

# Se muestran las filas que tengan al menos un valor superior a max_val
df_iris_outliers = df_iris_numeric[df_bool.sum(axis=1) > 0]
df_iris_outliers

,sepal_length,sepal_width,petal_length,petal_width
15,5.7,4.4,1.5,0.4
33,5.5,4.2,1.4,0.2


In [ ]:
max_val

2.5758293035489004